# utils 

> utility module for data_io, numerics, etc.
> for example, generating a pandas DataFrame for testing purpose

In [ ]:
#| default_exp utils 

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import pandas as pd
from datetime import datetime
from functools import reduce
from zoneinfo import ZoneInfo
from configparser import ConfigParser



In [ ]:
#| export
from data_io_nbdev.config.drivers import drivers_by_id
from data_io_nbdev.config.vehicles import trucks_by_id


## Generate state

In [ ]:
#| export
def generate_state(tz: ZoneInfo)->pd.DataFrame:
    """
    generate a pandas DataFrame for testing purpose
    """
    ts = pd.to_datetime(datetime.now(tz=tz))
    ts_ind = ts + pd.to_timedelta(np.arange(0, 4 * 20, 20), "ms")
    ts_ind = ts_ind.tz_localize(None).astype("datetime64[us]").tz_localize(tz)
    s = np.arange(12)
    a_1 = s[:4]
    a_2 = s[4:8]
    a_3 = s[8:]
    df_ss = pd.DataFrame(
        {"timestep": ts_ind, "velocity": a_1, "thrust": a_2, "brake": a_3}
    )  # .set_index('timestep')
    df_ss.columns.name = "qtuple"
    ui_t = df_ss.loc[:, ["velocity", "thrust"]]
    state = df_ss.stack().swaplevel(0, 1)
    state.name = "state"
    state.index.names = ["rows", "idx"]
    state.sort_index(inplace=True)
    return state

In [ ]:
#| hide
tz = trucks_by_id["VB7"].site.tz

state = generate_state(tz)
state['timestep'].dtype
state
# state['timestep'].dtype

In [ ]:

ts = pd.to_datetime(datetime.now())
ts_ind1 = ts + pd.to_timedelta(np.arange(0, 4 * 20, 20), "ms").astype("timedelta64[us]")
ts_ind = ts_ind1.astype('datetime64[us]')
ts_ind.unit

## Generate action

In [ ]:
#| export
def generate_action(tz: ZoneInfo)->pd.DataFrame:
    """
    generate a pandas DataFrame for testing purpose
    """

    s = np.arange(12)
    ts = pd.to_datetime(datetime.now(tz=tz))
    a = len(s) + np.arange(15)
    speed_ser = pd.Series(np.linspace(40, 60, 3), name="speed")
    row_array = a.reshape(3, 5).transpose()
    rows_df = pd.DataFrame(row_array)
    rows_df.columns = [f"r{i}" for i in np.arange(3)]
    ts_ind = ts + pd.to_timedelta(np.arange(5 * 20, 8 * 20, 20), "ms")
    ts_ind = ts_ind.tz_localize(None).astype("datetime64[us]").tz_localize(tz)
    ts_ser = pd.Series(ts_ind, name="timestep")
    throttle_ser = pd.Series(np.linspace(0, 1.0, 5), name="throttle")
    # throttle_ser
    dfs = [rows_df, ts_ser, speed_ser, throttle_ser]
    action = (
        reduce(
            lambda left, right: pd.merge(
                left, right, how="outer", left_index=True, right_index=True
            ),
            dfs,
        )
        .stack()
        .swaplevel(0, 1)
        .sort_index()
    )
    
    action.name = "action"
    action.index.names = ["rows", "idx"]
    return action

In [ ]:
#| hide
action = generate_action(tz=tz)
action

In [ ]:
action['timestep'].dtype

## Generate reward

In [ ]:
#| export

def generate_reward(tz: ZoneInfo)->pd.DataFrame:
    """
    generate a pandas DataFrame for testing purpose
    """
    ts = pd.to_datetime(datetime.now(tz=tz))
    ts_ind = ts + pd.to_timedelta(np.arange(0, 4 * 20, 20), "ms")
    ts_ind = ts_ind.tz_localize(None).astype("datetime64[us]").tz_localize(tz)
    s = np.arange(12)
    a = len(s) + np.arange(15)
    reward = (
        pd.DataFrame({"work": len(s) + len(a), "timestep": ts_ind[0]}, index=[0])
        .stack()
        .swaplevel(0, 1)
        .sort_index()
    )
    # reward_index = (reward.name,  ts_ind[0], 0)
    reward.index.names = ["rows", "idx"]
    reward.name = "reward"
    return reward


In [ ]:
#| hide
generate_reward(tz=tz)

## Generate nstate

In [ ]:
#| export
def generate_nstate(tz: ZoneInfo)->pd.DataFrame:
    """
    generate a pandas DataFrame for testing purpose
    """
    ts = pd.to_datetime(datetime.now(tz=tz))
    s = np.arange(12)
    a = len(s) + np.arange(15)
    reward = generate_reward(tz=tz)
    
    ts_ind = ts + pd.to_timedelta(5, "s") + pd.to_timedelta(np.arange(0, 4 * 20, 20), "ms")
    ts_ind = ts_ind.tz_localize(None).astype("datetime64[us]").tz_localize(tz)
    s = (
            np.arange(12) + len(s) + len(a) + len(reward) - 1
    )  # exclude the timestamp in reward
    a1 = s[:4]
    a2 = s[4:8]
    a3 = s[8:]
    nstate = (
        pd.DataFrame({"timestep": ts_ind, "velocity": a1, "thrust": a2, "brake": a3})
        # .set_index('timestamp')
        .stack()
        .swaplevel(0, 1)
        .sort_index()
    )
    nstate.name = "nstate"
    nstate.index.names = ["rows", "idx"]
    return nstate

In [ ]:
#| hide
generate_nstate(tz=tz)

## Generate observation

In [ ]:
#| export
def generate_observation(tz: ZoneInfo)-> list[pd.Series]:
    """
    generate a list of pandas Series for testing purpose
    """

    ts = pd.to_datetime(datetime.now(tz=tz))
    state = generate_state(tz)
    action = generate_action(tz)
    reward = generate_reward(tz)
    nstate = generate_nstate(tz)
    
    timestamp = pd.DatetimeIndex([ts], name="timestamp").tz_localize(None).astype(
        "datetime64[us]"
    ).tz_localize(tz)
    timestamp = pd.Series(timestamp, name="timestamp")
    timestamp.index = pd.MultiIndex.from_product(
        [timestamp.index, [0]], names=["rows", "idx"]
    )
    timestamp_index = (timestamp.name, "", 0)
    state_index = [(state.name, *i) for i in state.index]
    reward_index = [(reward.name, *i) for i in reward.index]
    action_index = [(action.name, *i) for i in action.index]
    nstate_index = [(nstate.name, *i) for i in nstate.index]
    
    multiindex = pd.MultiIndex.from_tuples(
        [timestamp_index, *state_index, *action_index, *reward_index, *nstate_index]
    )
    observation_list = [timestamp, state, action, reward, nstate]
    observation = pd.concat(observation_list)
    observation.index = multiindex
    observation0 = observation.copy()
    observation0.loc["timestamp", "", 0] = ts + pd.Timedelta(1, "h")
    observation1 = observation.copy()
    observation1.loc["timestamp", "", 0] = ts + pd.Timedelta(2, "h")
    observation2 = observation.copy()
    observation2.loc["timestamp", "", 0] = ts + pd.Timedelta(3, "h")
    observation3 = observation.copy()
    observation3.loc["timestamp", "", 0] = ts + pd.Timedelta(4, "h")
    observation4 = observation.copy()
    observation4.loc["timestamp", "", 0] = ts + pd.Timedelta(5, "h")
    observation_list = [
        observation0,
        observation1,
        observation2,
        observation3,
        observation4,
    ]
    return observation_list


In [ ]:
#| hide
generate_observation(tz=tz)

## Generate MultiIndex DataFrame

In [ ]:
#| export
def generate_df_multiindex(tz: ZoneInfo)->pd.DataFrame:

    observation_list = generate_observation(tz=tz)
    dfs_epi = pd.concat(observation_list, axis=1).transpose()
    dfs_epi.columns.names = ["qtuple", "rows", "idx"]
    # dfs_epi
    
    dfs_episode = dfs_epi.copy()
    dfs_episode.set_index(("timestamp", "", 0), inplace=True)
    dfs_episode.sort_index(axis=1, inplace=True)
    dfs_episode.index.name = "timestamp"
    idx = pd.IndexSlice
    state_cols_float = [("state", col) for col in ["brake", "thrust", "velocity"]]
    action_cols_float = [("action", col) for col in ["r0", "r1", "r2", "speed", "throttle"]]
    reward_cols_float = [("reward", "work")]
    nstate_cols_float = [("nstate", col) for col in ["brake", "thrust", "velocity"]]
    for col in action_cols_float + state_cols_float + reward_cols_float + nstate_cols_float:
        dfs_episode[col[0], col[1]] = dfs_episode[col[0], col[1]].astype(
            "float"
        )  # float16 not allowed in parquet
    return dfs_episode


In [ ]:
#| hide
generate_df_multiindex(tz=tz)

## prepend two levels of index "vehicle" and "driver" to the DataFrame object

In [ ]:
#|export
def generate_eos_df(tz: ZoneInfo)->pd.DataFrame:
    """
    generate a pandas DataFrame for testing purpose
    """
    dfs_episode = generate_df_multiindex(tz=tz)
    
    ts = pd.to_datetime(datetime.now(tz=tz))
    dfs_episode = pd.concat(
        [dfs_episode], keys=[drivers_by_id["wang-cheng"].pid], names=["driver"]
    )
    dfs_episode = pd.concat(
        [dfs_episode], keys=[trucks_by_id["VB7"].vid], names=["vehicle"]
    )
    episodestart = ts - pd.Timedelta(1, "h")
    dfs_episode = pd.concat([dfs_episode], keys=[ts], names=["episodestart"])
    dfs_episode = dfs_episode.swaplevel(1, 0, axis=0)
    dfs_episode = dfs_episode.swaplevel(1, 2, axis=0)
    dfs_episode.sort_index(inplace=True)
    return dfs_episode


In [ ]:
#|output: true
generate_eos_df(tz)

In [ ]:
assert isinstance(generate_eos_df(tz).index, pd.MultiIndex), f"dfs_episode.index is not a MultiIndex"

In [ ]:
from fastcore.test import *

In [ ]:
test_eq(isinstance(generate_eos_df(tz).index, pd.MultiIndex), True)

## Generate Recipe for local pool storage

In [ ]:
#| export
from data_io_nbdev.data.location import EosLocation, locations_by_abbr
from data_io_nbdev.data.core import configparser_as_dict, get_filemeta_config, ObservationMetaECU, StateSpecsECU, ActionSpecs, RewardSpecs 

In [ ]:
#| output: true
truck = trucks_by_id["VB7"]
meta = ObservationMetaECU(
    state_specs=StateSpecsECU(),
    action_specs=ActionSpecs(
        action_unit_code="nm",
        action_row_number=truck.torque_table_row_num_flash,
        action_column_number=truck.torque_table_col_num,
    ),
    reward_specs=RewardSpecs(reward_unit_code="wh", reward_number=1),
    site=locations_by_abbr[truck.site.abbr],
)

In [ ]:
#| output: true
meta

In [ ]:
#| output: true
data_folder = "data"
coll_type = "RECORD"
truck = trucks_by_id["default"]
truck.site = locations_by_abbr[truck.site.abbr]


In [ ]:
#|output: true
number_states, number_actions = meta.get_number_of_states_actions()
recipe_default: ConfigParser = ConfigParser()
recipe_default.read_dict(
    {
        "DEFAULT": {  # should go into parquet tabel meta info
            "data_folder": data_folder,  # '.',
            "recipe_file_name": "recipe.ini",  # 'recipe.ini',
            "coll_type": coll_type,
        },
        "array_specs": {  # should go into parquet columns meta info
            "states": str(number_states),  # 50*4*3
            "actions": str(number_actions),  # 17*4
            "rewards": "1",
            "next_states": str(number_states),  # 50*4*3
        },
    }
)


In [ ]:
#|output: true
recipe_generated = get_filemeta_config(data_folder="data",config_file="recipe.ini",meta=meta, coll_type="RECORD")

In [ ]:
#| output: true
configparser_as_dict(recipe_generated)

In [ ]:
test_eq(configparser_as_dict(recipe_default), configparser_as_dict(recipe_generated))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()
